In [1]:
#! python3 -m pip install --upgrade bokeh
#! python3 -m pip install --upgrade jupyter_bokeh


In [1]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
# Plot a complex chart with interactive hover in a few lines of code

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)

group = df.groupby(by=['cyl', 'mfr'])
source = ColumnDataSource(group)

p = figure(width=800, height=300, title="Mean MPG by # Cylinders and Manufacturer",
           x_range=group, toolbar_location=None, tools="")

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Manufacturer grouped by # Cylinders"
p.xaxis.major_label_orientation = 1.2

index_cmap = factor_cmap('cyl_mfr', palette=['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c'], 
                         factors=sorted(df.cyl.unique()), end=1)

p.vbar(x='cyl_mfr', top='mpg_mean', width=1, source=source,
       line_color="white", fill_color=index_cmap, 
       hover_line_color="darkgrey", hover_fill_color=index_cmap)

p.add_tools(HoverTool(tooltips=[("MPG", "@mpg_mean"), ("Cyl, Mfr", "@cyl_mfr")]))

show(p)

In [3]:
import rbot

In [4]:
bn = rbot.Market.open("BN", "BTCBUSD")

In [5]:
ohlcv = bn.ohlcv(rbot.NOW()-rbot.DAYS(10), 0, 60*60)

In [6]:
ohlcv

,open,high,low,close,volume,count
timestamp,,,,,,
2022-11-29 16:00:00+00:00,16376.95,16420.00,16336.00,16405.67,5878.50170,100607.0
2022-11-29 17:00:00+00:00,16405.40,16421.60,16352.00,16373.07,4688.58391,79724.0
2022-11-29 18:00:00+00:00,16373.09,16429.99,16350.30,16414.42,5029.87184,79155.0
2022-11-29 19:00:00+00:00,16414.11,16474.36,16411.07,16420.50,5114.47198,80671.0
2022-11-29 20:00:00+00:00,16421.25,16512.10,16416.73,16489.89,5673.13953,92573.0
...,...,...,...,...,...,...
2022-12-06 19:00:00+00:00,16972.79,16986.85,16925.73,16971.00,5663.63236,85563.0
2022-12-06 20:00:00+00:00,16971.00,16995.60,16952.33,16989.60,5561.50498,79311.0
2022-12-06 21:00:00+00:00,16989.31,17010.16,16975.77,16993.75,4055.02358,56417.0


In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
inc = ohlcv.open < ohlcv.close
dec = ohlcv.close < ohlcv.open 

In [9]:
delta = (ohlcv[1:2].index - ohlcv[0:1].index)[0]

In [10]:
w = delta.total_seconds() * 1_000

In [30]:
df

,open,high,low,close,volume,count
timestamp,,,,,,
2022-11-29 16:00:00+00:00,16376.95,16420.00,16336.00,16405.67,5878.50170,100607.0
2022-11-29 17:00:00+00:00,16405.40,16421.60,16352.00,16373.07,4688.58391,79724.0
2022-11-29 18:00:00+00:00,16373.09,16429.99,16350.30,16414.42,5029.87184,79155.0
2022-11-29 19:00:00+00:00,16414.11,16474.36,16411.07,16420.50,5114.47198,80671.0
2022-11-29 20:00:00+00:00,16421.25,16512.10,16416.73,16489.89,5673.13953,92573.0
...,...,...,...,...,...,...
2022-12-06 19:00:00+00:00,16972.79,16986.85,16925.73,16971.00,5663.63236,85563.0
2022-12-06 20:00:00+00:00,16971.00,16995.60,16952.33,16989.60,5561.50498,79311.0
2022-12-06 21:00:00+00:00,16989.31,17010.16,16975.77,16993.75,4055.02358,56417.0


In [47]:
df_inc = df[(df['open'] <= df['close'])]
df_dec = df[(df['close'] < df['open'])] 

In [45]:
df_inc

,open,high,low,close,volume,count
timestamp,,,,,,
2022-11-29 16:00:00+00:00,16376.95,16420.00,16336.00,16405.67,5878.50170,100607.0
2022-11-29 18:00:00+00:00,16373.09,16429.99,16350.30,16414.42,5029.87184,79155.0
2022-11-29 19:00:00+00:00,16414.11,16474.36,16411.07,16420.50,5114.47198,80671.0
2022-11-29 20:00:00+00:00,16421.25,16512.10,16416.73,16489.89,5673.13953,92573.0
2022-11-29 22:00:00+00:00,16460.14,16494.07,16446.27,16488.44,2943.39422,46640.0
...,...,...,...,...,...,...
2022-12-06 16:00:00+00:00,16985.66,17020.65,16969.03,16987.54,7367.12896,112652.0
2022-12-06 18:00:00+00:00,16961.06,17000.00,16912.00,16972.71,8385.22625,115452.0
2022-12-06 20:00:00+00:00,16971.00,16995.60,16952.33,16989.60,5561.50498,79311.0


In [46]:
df_dec

,open,high,low,close,volume,count
timestamp,,,,,,
2022-11-29 17:00:00+00:00,16405.40,16421.60,16352.00,16373.07,4688.58391,79724.0
2022-11-29 21:00:00+00:00,16490.28,16494.34,16447.01,16460.31,3240.18110,50468.0
2022-11-29 23:00:00+00:00,16488.45,16520.00,16420.10,16436.17,3862.58259,65255.0
2022-11-30 03:00:00+00:00,16955.74,16973.38,16834.00,16837.85,7468.62356,133569.0
2022-11-30 05:00:00+00:00,16873.24,16909.56,16858.19,16859.57,5042.12263,89762.0
...,...,...,...,...,...,...
2022-12-06 13:00:00+00:00,16997.18,17033.96,16976.83,16986.67,5042.68532,93821.0
2022-12-06 14:00:00+00:00,16986.70,17019.48,16960.33,16973.93,8218.37201,138193.0
2022-12-06 17:00:00+00:00,16987.54,17006.75,16946.36,16961.06,6610.89772,104801.0


In [107]:
ds = ColumnDataSource(ohlcv)

df_inc = ColumnDataSource(ohlcv[(ohlcv['open'] <= ohlcv['close'])])
df_dec = ColumnDataSource(ohlcv[(ohlcv['close'] < ohlcv['open'])])

delta = (ohlcv[1:2].index - ohlcv[0:1].index)[0]
w = delta.total_seconds() * 1_000 * 0.8

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"


p = figure(x_axis_type="datetime", tools=TOOLS, width=900, height=400,
           title="BTC chart", background_fill_color="#efefef")
   

p.xaxis.major_label_orientation = 0.8 # radians

#p.segment('index', 'high', 'index', 'low', source=ds, color="black")

p.segment('timestamp', 'high', 'timestamp', 'low', source=ds)

vbar_inc = p.vbar('timestamp', w, 'open', 'close', source=df_inc)
vbar_dec = p.vbar('timestamp', w, 'close', 'open', source=df_dec)

#bar_dec = p.vbar(df.index[dec], w, df.open[dec], df.close[dec], color="#eb3c40")
#bar_inc = p.vbar(df.index[inc], w, df.open[inc], df.close[inc], fill_color="white",
#       line_color="#49a3a3", line_width=2)



hover_inc = HoverTool(
    renderers=[vbar_inc],
    tooltips = [
           ("timestamp", "@timestamp{%F %R.%S}"),
           ("open", "@open{0.0}"),
           ("high", "@high{0.0}"),
           ("low", "@low{0.0}"),
           ("close", "@close{0.0}")
    ],
    formatters= {
        "@timestamp": "datetime",
    },
    mode="vline",
    show_arrow=False,
)       
hover_dec = HoverTool(
    renderers=[vbar_dec],
    tooltips = [
           ("timestamp", "@timestamp{%F %R.%S}"),
           ("open", "@open{0.0}"),
           ("high", "@high{0.0}"),
           ("low", "@low{0.0}"),
           ("close", "@close{0.0}")
    ],
    formatters= {
        "@timestamp": "datetime"
    },
    mode="vline",
    show_arrow=False,
)      

p.add_tools(hover_inc)
p.add_tools(hover_dec)

show(p)

In [62]:


def draw_ohlcv(p, ohlcv):
    ds = ColumnDataSource(ohlcv)

    df_inc = ColumnDataSource(ohlcv[(ohlcv['open'] <= ohlcv['close'])])
    df_dec = ColumnDataSource(ohlcv[(ohlcv['close'] < ohlcv['open'])])

    delta = (ohlcv[1:2].index - ohlcv[0:1].index)[0]
    w = delta.total_seconds() * 1_000 * 0.8



    p.segment('timestamp', 'high', 'timestamp', 'low', source=ds, color="#eb3c40")
    vbar_inc = p.vbar('timestamp', w, 'open', 'close', source=df_inc, fill_color="white", line_color="#49a3a3", line_width=2)
    vbar_dec = p.vbar('timestamp', w, 'close', 'open', source=df_dec)


    hover_inc = HoverTool(
        renderers=[vbar_inc],
        tooltips = [
           ("timestamp", "@timestamp{%F %R.%S}"),
           ("open", "@open{0.0}"),
           ("high", "@high{0.0}"),
           ("low", "@low{0.0}"),
           ("close", "@close{0.0}")
        ],
        formatters= {
            "@timestamp": "datetime",
        },
        mode="vline",
        show_arrow=False,
    )       

    hover_dec = HoverTool(
        renderers=[vbar_dec],
        tooltips = [
           ("timestamp", "@timestamp{%F %R.%S}"),
           ("open", "@open{0.0}"),
           ("high", "@high{0.0}"),
           ("low", "@low{0.0}"),
           ("close", "@close{0.0}")
        ],
        formatters= {
            "@timestamp": "datetime"
        },
        mode="vline",
        show_arrow=False,
    )      

    p.add_tools(hover_inc)
    p.add_tools(hover_dec)
    
    
    
    

In [112]:
def draw_price_line(p, ohlcv):
    p.line(x=ohlcv.index, y=ohlcv['close'])

In [115]:
from bokeh.layouts import column 
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure, show
from bokeh.core.properties import Instance


TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

WIDTH=900

dates = np.array(ohlcv.index, dtype=np.datetime64)

p.xaxis.major_label_orientation = 0.8 # radians
p = figure(x_axis_type="datetime", tools=TOOLS, width=WIDTH, height=400,
           title="BTC chart", background_fill_color="#efefef", x_range=(dates[0], dates[-1]))

draw_ohlcv(p, ohlcv)

volume = figure(x_axis_type="datetime", tools=TOOLS, width=WIDTH, height=100,
        title="BTC vol", background_fill_color="#efefef", x_range=p.x_range)

volume.line(x=ohlcv.index, y=ohlcv['volume'])

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                height=130, width=WIDTH, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

draw_price_line(select, ohlcv)

select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, volume, select))



/var/folders/5r/xmxmz7gj2cjcb7d65tgwvrt40000gn/T/ipykernel_5012/3261770516.py:11: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  dates = np.array(ohlcv.index, dtype=np.datetime64)


In [ ]:
fig = new_

In [85]:
p.x_range


DataRange1d(id='p8684', ...)

In [86]:
p.x_range

DataRange1d(id='p8684', ...)

In [184]:
from collections import OrderedDict

class Chart:
    def __init__(self, width, height, ohlc):
        self.figure = OrderedDict()
        self.width = width
        self.height = height
        self.x_range = None
        self.select = None


        # setup main figure
        dates = np.array(ohlcv.index, dtype=np.datetime64)        
        main = figure(x_axis_type="datetime", tools=TOOLS, width=WIDTH, height=height,
           title="BTC chart", background_fill_color="#efefef", x_range=(dates[0], dates[-1]))
        
        self.x_range = main.x_range
        self.figure['main'] = main
        
        self.draw_ohlcv(main, ohlcv)

        # setup select figure
        select = figure(title="Drag the middle and edges of the selection box to change the range above",
                height= int(height/4), width=WIDTH, y_range=main.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")            

        self.select = select

        self.draw_price_line(select, ohlcv)
        
        range_tool = RangeTool(x_range=self.x_range)
        range_tool.overlay.fill_color = "navy"
        range_tool.overlay.fill_alpha = 0.2

        select.ygrid.grid_line_color = None
        select.add_tools(range_tool)
        select.toolbar.active_multi = range_tool
        

    
    def new_figure(self, name, height, title):
        p = figure(x_axis_type="datetime", width=WIDTH, height=height, tools="", toolbar_location=None,
            title=title, background_fill_color="#efefef", x_range=self.x_range)
        self.figure[name] = p 
        
    def get_figure(self, name):
        return self.figure[name]

    def show(self):
        figure = []
        for key in self.figure:
            print(key)
            figure.append(self.figure[key])

        figure.append(self.select)
            
        show(column(figure))

    def draw_ohlcv(self, p, ohlcv):
        ds = ColumnDataSource(ohlcv)

        df_inc = ColumnDataSource(ohlcv[(ohlcv['open'] <= ohlcv['close'])])
        df_dec = ColumnDataSource(ohlcv[(ohlcv['close'] < ohlcv['open'])])

        delta = (ohlcv[1:2].index - ohlcv[0:1].index)[0]
        w = delta.total_seconds() * 1_000 * 0.8

        p.segment('timestamp', 'high', 'timestamp', 'low', source=ds, color="#eb3c40")
        vbar_inc = p.vbar('timestamp', w, 'open', 'close', source=df_inc, fill_color="white", line_color="#49a3a3", line_width=2)
        vbar_dec = p.vbar('timestamp', w, 'close', 'open', source=df_dec)

        hover_inc = HoverTool(
            renderers=[vbar_inc],
            tooltips = [
                ("timestamp", "@timestamp{%F %R.%S}"),
                ("open", "@open{0.0}"),
                ("high", "@high{0.0}"),
                ("low", "@low{0.0}"),
                ("close", "@close{0.0}")
            ],
            formatters= {
                "@timestamp": "datetime",
            },
            mode="vline",
            show_arrow=False,
        )       

        hover_dec = HoverTool(
            renderers=[vbar_dec],
            tooltips = [
                ("timestamp", "@timestamp{%F %R.%S}"),
                ("open", "@open{0.0}"),
                ("high", "@high{0.0}"),
                ("low", "@low{0.0}"),
                ("close", "@close{0.0}")
            ],
            formatters= {
                "@timestamp": "datetime"
            },
            mode="vline",
            show_arrow=False,
        )      

        p.add_tools(hover_inc)
        p.add_tools(hover_dec)

    def draw_price_line(self, p, ohlc):
        p.line(x=ohlcv.index, y=ohlcv['close'])
        
    def draw_volume_line(self, name, ohlcv):
        p = self.figure[name]
        p.line(x=ohlcv.index, y=ohlcv['volume'])        


In [185]:
chart = Chart(800, 400, ohlcv)

chart.new_figure('volume', 100, 'trading volume in btc')
chart.draw_volume_line('volume', ohlcv)



/var/folders/5r/xmxmz7gj2cjcb7d65tgwvrt40000gn/T/ipykernel_5012/1436718999.py:13: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  dates = np.array(ohlcv.index, dtype=np.datetime64)


In [186]:
chart.show()

main
volume
